# Qiskit serverless demonstration: Hamiltonian simulation template

### Convert notebook to a Qiskit Serverless program

Here we use `nbconvert` to convert the Jupyter notebook to a Python program (`.py` file).

In [1]:
from pathlib import Path
from nbconvert.exporters import ScriptExporter

notebook_filename = "hamsim_template.ipynb"
source_path = Path("./source_files/")
source_path.mkdir(exist_ok=True)
script, script_info = ScriptExporter().from_filename(notebook_filename)
script_name = script_info["metadata"]["name"]
script_file = source_path / (script_name + script_info["output_extension"])
with script_file.open("w") as f:
    f.write(script)
print(f"Notebook saved to {script_file}")

KeyboardInterrupt: 

In [ ]:
with script_file.open() as f:
    print("".join(f.readlines()[:25]), "\n[...]")

### Upload template to serverless

In [ ]:
from qiskit_ibm_catalog import QiskitServerless, QiskitFunction

title = script_file.stem

template = QiskitFunction(
    title=script_file.stem,
    entrypoint=script_file.name,
    working_dir=str(source_path),
    dependencies=[
        "qiskit-addon-utils==0.1.0",
        "qiskit-addon-aqc-tensor[quimb-jax]==0.1.0",
        "mergedeep",
    ],
)

serverless = QiskitServerless()
serverless.upload(template)
print(f"Uploaded program: {title}")

In [ ]:
serverless.list()

### Load the template

In [ ]:
template = serverless.load(title)

### Prepare hamiltonian and observable

In [ ]:
from qiskit.quantum_info import SparsePauliOp

hamiltonian = SparsePauliOp.from_sparse_list(
    [("XX", (i, i + 1), 1.0) for i in range(3)], num_qubits=4
) + SparsePauliOp.from_sparse_list(
    [("YY", (i, i + 1), 1.0) for i in range(3)], num_qubits=4
)
observable = SparsePauliOp.from_sparse_list([("ZZ", (1, 2), 1.0)], num_qubits=4)

### Run the template

In [ ]:
job = template.run(
    dry_run=True,
    hamiltonian=hamiltonian,
    observable=observable,
    backend_name="ibm_fez",
    estimator_options={},
    aqc_evolution_time=0.2,
    aqc_ansatz_num_trotter_steps=1,
    aqc_target_num_trotter_steps=32,
    remainder_evolution_time=0.2,
    remainder_num_trotter_steps=4,
    aqc_max_iterations=300,
)

In [ ]:
print(job.job_id)

### Check the logs (this will return any info about how AQC is behaving)

In [ ]:
job.status()

In [ ]:
job.logs()

### Get the final result (expectation value)

In [ ]:
job.result()